###Install Requirements

In [1]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 14400, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 14400 (delta 14), reused 19 (delta 10), pack-reused 14371
Receiving objects: 100% (14400/14400), 13.61 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (9915/9915), done.
/content/yolov5
     |████████████████████████████████| 182 kB 4.3 MB/s 
     |████████████████████████████████| 62 kB 672 kB/s 
     |████████████████████████████████| 1.6 MB 47.9 MB/s 
     |████████████████████████████████| 42 kB 169 kB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 145 kB 16.2 MB/s 
     |████████████████████████████████| 138 kB 51.5 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 178 kB 51.1 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
Setup complete. Using torch 1.13.0+cu116 (Tesla T4)


In [2]:
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=e0f702187ec205b0400286d1589b8e930389f2495bbe95c830179b161b0ee862
  Stored in directory: /root/.cache/pip/wheels/9e/fc/1c/cf980e6413d3ee8e70cd8f39e2366b0f487e3e221aeb452eb0
Successfully built python-multipart


In [3]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 5.3 MB/s 
     |████████████████████████████████| 2.3 MB 53.0 MB/s 
     |████████████████████████████████| 55 kB 4.7 MB/s 
     |████████████████████████████████| 213 kB 53.9 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |████████████████████████████████| 84 kB 1.2 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 278 kB 53.6 MB/s 
     |████████████████████████████████| 106 kB 8.7 MB/s 
     |████████████████████████████████| 84 kB 5.4 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 80 kB 6.8 MB/s 
     |████████████████████████████████| 68 kB 3.4 MB/s 
     |████████████████████████████████| 68 kB 8.0 MB/s 
     |████████████████████████████████| 68 kB 8.4 MB/s 
     |████████████████████████████████| 68 kB 7.6 MB

###Import Library

In [9]:
# importing cv2 
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import shutil


In [10]:

import requests
from PIL import Image
from io import BytesIO

In [11]:
import gradio as gr

In [12]:
import tensorflow as tf

In [13]:
import re

#Functions

In [16]:
def createDirectory(path):
  # checking if the directory demo_folder 
  # exist or not
  if not os.path.exists(path):
    # if the demo_folder directory is not present 
    # then create it
    os.makedirs(path)

def removeDirectory(path):
  # checking if the directory demo_folder 
  # exist or not
  if os.path.exists(path):
    # if the demo_folder directory is present 
    # then remove it
    shutil.rmtree(path)


In [17]:
reObjLabel = re.compile(r"(?P<label>\S+) (?P<center_x>\S+) (?P<center_y>\S+) (?P<w>\S+) (?P<h>\S+) (?P<percent>\S+)")
def readLabels(filename, width, height):
  f = open(filename, "r")
  lst_x0 = []
  lst_y0 = []
  lst_x1 = []
  lst_y1 = []
  lst_per = []
  for line in f:
    reLabel = reObjLabel.match(line)
    center_x = int(float(reLabel.group('center_x'))*width)
    center_y = int(float(reLabel.group('center_y'))*height)
    w = int(float(reLabel.group('w'))*width)
    h = int(float(reLabel.group('h'))*height)
    x = int(center_x - w/2)
    y = int(center_y - h/2)
    lst_x0.append(x)
    lst_y0.append(y)
    lst_x1.append(x + w)
    lst_y1.append(y + h)
    lst_per.append(float(reLabel.group('percent')))
  f.close()
  dict_obj_detect = {'x0':lst_x0, 'y0':lst_y0, 'x1':lst_x1, 'y1':lst_y1, 'percent':lst_per }
  return dict_obj_detect


def cutImage(img, dict_obj_IO, width, height):
  size_ = len(dict_obj_IO['x0'])

  lst_img = []
  for i in range(size_):
    x0 = dict_obj_IO['x0'][i]
    y0 = dict_obj_IO['y0'][i]
    x1 = dict_obj_IO['x1'][i]
    y1 = dict_obj_IO['y1'][i]
    percent_detection = dict_obj_IO['percent'][i]
    if (percent_detection > .85):
      tmp_img = img[y0:y1, x0:x1]
      tmp_img = cv2.resize(tmp_img, (width, height))
      lst_img.append(tmp_img)
  return np.array(lst_img)

#Calibration

In [43]:
SIZE_ROW_IMG = 416

DECT_OBJ_IMG_WIDTH = 110
DECT_OBJ_IMG_HEIGHT = 20
DECT_OBJ_IMG_CHANNEL = 3

SCORE_DETECTION_CLASS_NAME = np.array(['0', '1', '2', '3', '4', '5', '6', 'L'])


In [36]:
dict_score =     {'0':0, 
                  '1':1,
                  '2':2, 
                  '3':3, 
                  '4':4, 
                  '5':5, 
                  '6':6, 
                  'L':0}

#Init

In [24]:

  createDirectory('/content/row')

In [25]:
def get_clasifier_model():
  shutil.unpack_archive("/content/model.zip", '/content/model', "zip")
  #tf.config.run_functions_eagerly(True)
  model = tf.keras.models.load_model('/content/model/score_detect_classifier')
  model.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)
  return model

In [26]:
model_clasifier = get_clasifier_model()


#Run Inference  With Trained Weights

In [20]:
def score_detection():
  !python detect.py --weights /content/best.pt --img {SIZE_ROW_IMG} --conf 0.1 --source /content/row --save-conf --save-txt

In [37]:
def score_clasifier(filename_img, filename_pred):
  dict_obj_detect = readLabels(filename_pred, SIZE_ROW_IMG, SIZE_ROW_IMG)
  row_img = cv2.imread(filename_img)
  lst_detect_obj = cutImage(row_img, dict_obj_detect, DECT_OBJ_IMG_WIDTH, DECT_OBJ_IMG_HEIGHT)
  pred_val = model_clasifier.predict(lst_detect_obj)
  pred_val = [np.argmax(pred_vec) for pred_vec in pred_val]
  return pred_val

  

In [45]:
def predict_score(input_img):
  removeDirectory('/content/yolov5/runs/detect/exp')
  filename = '/content/row/name.png'
  try:
    input_img = cv2.resize(input_img, (SIZE_ROW_IMG, SIZE_ROW_IMG))
    cv2.imwrite(filename, input_img)
  except Exception:
    raise Exception('Error', 'Image ')
  score_detection()
  filename_pred = '/content/yolov5/runs/detect/exp/labels/name.txt'
  pred_class = score_clasifier(filename, filename_pred)
  pred_score = SCORE_DETECTION_CLASS_NAME[pred_class]
  print(pred_score)
  sum = 0
  for score_ in pred_score:
    sum += dict_score[score_]
  str_score = "score: {}, total score: {}".format(' '.join(pred_score), str(sum))
  return str_score

demo = gr.Interface(predict_score, gr.Image(), "text")
demo.launch(debug=True)
#demo.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

detect: weights=['/content/best.pt'], source=/content/row, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-27-g454dae1 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/row/name.png: 416x416 8 score_evals, 8.4ms
Speed: 0.4ms pre-process, 8.4ms inference, 1.6ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp
1 labels saved to runs/detect/exp/labels
1/1 [==============================] - 0s 27ms/step
['4' '2' '3' '0' '0' '0' '4']
Keyboard interruption in main thread... closing server.
